# 광주은행 KJ7
#### 데이터분석부(개인신용대출분석팀) 최대훈 대리
#### 수도권전략부 이건호 대리
##### 목표 : 과거 구매 이력과 현재 구매 정보를 활용하여, 과거 구매 상품 중 현재 재구매 할 만한 상품을 제안 (예측)

In [79]:
from jbfg.pandas_utils import *

# pd.set_option('display.max_columns',None)  # 칼럼 전체 표시

# pd.set_option('display.max_rows',None)      # row 전체 표시

In [82]:
# 광주은행 데이터셋
schema = 'l2_kjbk_extnl'

In [83]:
# 100개 상품정보
sql1 = 'SELECT * FROM products_info'

In [84]:
df_products_info = get_df(sql1, schema)

In [85]:
df_products_info.tail()

,product_id,name,category_id,division_id,category_name,division_name
95,47209,Organic Hass Avocado,24,4,fresh fruits,produce
96,21903,Organic Baby Spinach,123,4,packaged vegetables fruits,produce
97,21137,Organic Strawberries,24,4,fresh fruits,produce
98,13176,Bag of Organic Bananas,24,4,fresh fruits,produce
99,24852,Banana,24,4,fresh fruits,produce


In [36]:
# sql = 'SELECT DISTINCT product_id FROM products_info'
# df_products_info = get_df(sql, schema)
# df_products_info

In [91]:
# 5,859명 고객 과거 구매 이력
sql2 = "SELECT * FROM train_x_past "
df_train_x_past = get_df(sql2, schema)

In [92]:
df_train_x_past.head()
# ROWS 1253972

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id
0,717029,196613,15,1,9,3.0,5,0,41220
1,2325772,196613,35,6,11,2.0,7,1,41220
2,642668,196613,69,2,16,2.0,11,1,4210
3,1326515,196613,61,3,19,3.0,3,1,4210
4,1437845,196613,36,3,15,4.0,16,0,4210


In [93]:
sql2 = """
         SELECT *
         FROM train_x_past
         where 1=1
        -- cstno IN ('196613', '32775', '32780','32805','163819')
         AND is_reordered = '1'
         AND add_order = '1'
        -- AND order_interval = '15.0'
       """
df_train_x_past2 = get_df(sql2, schema)
df_train_x_past2

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id
0,185079,196613,94,6,17,6.0,1,1,38689
1,358293,196613,93,0,11,4.0,1,1,38689
2,2919366,196613,89,4,10,3.0,1,1,38689
3,1933676,196613,84,5,11,0.0,1,1,26604
4,2503785,196613,52,1,8,3.0,1,1,44359
...,...,...,...,...,...,...,...,...,...
119329,1218210,163819,13,1,11,6.0,1,1,24852
119330,1471330,163819,7,1,10,12.0,1,1,24852
119331,1714563,163819,3,1,16,13.0,1,1,24852
119332,2549215,163819,9,1,11,7.0,1,1,24852


In [94]:
# # cstno 리스트
# sql = "SELECT distinct cstno FROM train_x_past"
# df_train_x_past = get_df(sql, schema)
# df_train_x_past

In [104]:
# 상품 ID를 제외한 5,859명 고객들의 현재 구매정보
sql3 = 'SELECT * FROM train_x_current'
df_train_x_current = get_df(sql3, schema)
df_train_x_current

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered
0,806422,41116,47,5,18,3.0,4,1
1,2146033,127113,45,1,10,7.0,24,1
2,3055498,140168,45,5,10,26.0,11,1
3,3267527,60604,52,0,16,8.0,2,1
4,3125992,136554,46,4,9,5.0,2,0
...,...,...,...,...,...,...,...,...
24419,3184000,174046,100,5,7,2.0,12,1
24420,1754722,6710,100,5,11,3.0,21,1
24421,1076973,150840,100,0,13,0.0,18,1
24422,1247049,99707,100,1,13,3.0,6,1


In [95]:
sql33 = "SELECT * FROM train_x_current where cstno = '196613'"
df_train_x_current3 = get_df(sql33, schema)
df_train_x_current3

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered
0,2414402,196613,100,6,13,5.0,4,1
1,2414402,196613,100,6,13,5.0,1,1


In [97]:
df_train_x_current.tail()
# ROWS 24424

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered
24419,3184000,174046,100,5,7,2.0,12,1
24420,1754722,6710,100,5,11,3.0,21,1
24421,1076973,150840,100,0,13,0.0,18,1
24422,1247049,99707,100,1,13,3.0,6,1
24423,1775787,201038,100,5,9,2.0,1,1


In [98]:
# 5,859명 고객 현재 주문 정보에 대한 상품 ID
sql4 = 'SELECT * FROM train_y'
df_train_y = get_df(sql4, schema)
df_train_y.tail()
# ROWS 24424

,product_id
24419,24852
24420,24852
24421,24852
24422,24852
24423,24852


In [99]:
# 4,141명 고객들의 과거 구매 이력
sql5 = 'SELECT * FROM test_x_past'
df_test_x_past = get_df(sql5, schema)
df_test_x_past.tail()
# ROWS 541922

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered,product_id
541917,2672845,119134,39,1,14,17.0,4,1,24852
541918,2731678,119134,38,5,14,3.0,4,1,24852
541919,2860406,119134,9,5,9,8.0,1,1,24852
541920,3252013,119134,4,5,10,7.0,2,1,24852
541921,3275894,119134,16,4,10,3.0,10,1,24852


In [100]:
# 상품 ID를 제외한 4,141명 고객들의 현재 구매정보
sql6 = 'SELECT * FROM test_x_current'
df_test_x_current = get_df(sql6, schema)
df_test_x_current.tail()
# ROWS 16126

,order_id,cstno,order_number,week_day,order_hour,order_interval,add_order,is_reordered
16121,2047884,57077,46,5,15,8.0,2,1
16122,567625,83592,43,4,12,0.0,1,1
16123,596442,157740,49,6,10,3.0,2,1
16124,671102,92043,43,4,10,10.0,1,1
16125,529654,62871,58,4,8,8.0,22,1


## 접근방법 1
* 상품별 구매횟수 많은 것을 찾아보자 

In [101]:
# 과거 5,859명 상품별 구매횟수
past_data = """
            WITH 구매횟수 AS(
                                                     --- 구매횟수가 가장 많은 순서대로 리스트 추출
               select product_id AS 상품ID,
                      count(product_id) AS 구매횟수
               from train_x_past
               group by product_id
               order by 구매횟수 DESC  
            )
            
               SELECT *
               FROM 구매횟수
               
            """
df_past_data = get_df(past_data, schema)
df_past_data.head(10)

,상품id,구매횟수
0,46979,9475
1,29487,3829
2,30489,12781
3,19660,9424
4,44142,4405
5,8424,5966
6,19057,16387
7,38689,8837
8,20995,5343
9,40706,14452


In [103]:
# 과거 5,859명 데이터 중에 구매횟수가 가장 많은 상위 10개 품목
# fresh fruits 의 비중이 많았다!
past_top10 = """
               select
                       a.product_id    AS 상품id,
                       b.name          AS 상품명,
                       b.category_name AS 분류,
                       count(a.product_id) AS 구매횟수
               from train_x_past a LEFT JOIN products_info b             -- 과거 구매이력에 상품명 붙이기위함
                                   ON a.product_id = b.product_id
               group by a.product_id, b.name, b.category_name
               order by 구매횟수 desc
               
            """
df_past_top10 = get_df(past_top10, schema)
# df_past_top10.head(10)
df_past_top10

,상품id,상품명,분류,구매횟수
0,24852,Banana,fresh fruits,74686
1,13176,Bag of Organic Bananas,fresh fruits,67514
2,21137,Organic Strawberries,fresh fruits,52917
3,47209,Organic Hass Avocado,fresh fruits,42604
4,21903,Organic Baby Spinach,packaged vegetables fruits,40112
...,...,...,...,...
95,34358,Garlic,fresh vegetables,4210
96,47144,Unsweetened Original Almond Breeze Almond Milk,soy lactosefree,4122
97,29487,Roma Tomato,fresh vegetables,3829
98,23909,2% Reduced Fat Milk,milk,3547


In [18]:
# 과거 4,141명은 ?
past_4141_top10 = """
               select
                       a.product_id    AS 상품id,
                       b.name          AS 상품명,
                       b.category_name AS 분류,
                       count(a.product_id) AS 구매횟수
               from test_x_past a LEFT JOIN products_info b             -- 과거 구매이력에 상품명 붙이기위함
                                   ON a.product_id = b.product_id
               group by a.product_id, b.name, b.category_name
               order by 구매횟수 desc
               
            """
df_past_4141_top10 = get_df(past_4141_top10, schema)
df_past_top10.head(10)
# df_past_4141_top10

,상품id,상품명,분류,구매횟수
0,24852,Banana,fresh fruits,74686
1,13176,Bag of Organic Bananas,fresh fruits,67514
2,21137,Organic Strawberries,fresh fruits,52917
3,47209,Organic Hass Avocado,fresh fruits,42604
4,21903,Organic Baby Spinach,packaged vegetables fruits,40112
5,27845,Organic Whole Milk,milk,29757
6,27966,Organic Raspberries,packaged vegetables fruits,28207
7,47766,Organic Avocado,fresh fruits,25046
8,26209,Limes,fresh fruits,21216
9,47626,Large Lemon,fresh fruits,21012


In [105]:
# # ROWS 24424
# train_x_current = """
#                 SELECT *
#                 FROM train_x_current
#                 """
# df_train_x_current = get_df(train_x_current, schema)
# df_train_x_current

In [106]:
# # ROWS 24424
# train_y = """
#           SELECT *
#           FROM train_y
#           """
# df_train_y = get_df(train_y, schema)
# df_train_y

In [107]:
# # 5,859명 현재 구매 정보
# train_current = pd.concat([df_train_x_current, df_train_y], axis=1)
# train_current

# 접근방법 2
# 1. 재구매율
- 고객별 재구매율에 따라 구매 상품이 달라지지 않을까?
- 구매상품 개수 / 재구매상품 개수 / 재구매상품 카테고리 / 빈도 등

# 2. 직전 구매 일자 간격
- 구매 간격에 따라 구매상품이 달라진다?


# 조건/경우의수/상관관계/함수모듈화
- 고객전체에 대한 전반적인 분석 및 시각화 ---> 모델링 방향성 갖추기
- 경우의수, 조건 적용하여 고객별 구매상품 예측

In [115]:
# 재구매상품들은 뭐가 많을까?
# cstno = '196613'
re_ordered =  """
               select
                       a.product_id    AS 상품id,
                       b.name          AS 상품명,
                       b.category_name AS 분류,
                       count(a.product_id) AS 구매횟수
               from train_x_past a LEFT JOIN products_info b             -- 과거 구매이력에 상품명 붙이기위함
                                   ON a.product_id = b.product_id
               where 1=1
             --  and is_reordered = '1'
             --  and cstno ='196613'
               and cstno ='32775'
               group by a.product_id, b.name, b.category_name
               order by 구매횟수 desc
               
            """
df_re_ordered = get_df(re_ordered, schema)
df_re_ordered

,상품id,상품명,분류,구매횟수
0,38689,Organic Reduced Fat Milk,milk,29
1,34358,Garlic,fresh vegetables,20
2,30489,Original Hummus,fresh dips tapenades,17
3,21137,Organic Strawberries,fresh fruits,13
4,10749,Organic Red Bell Pepper,fresh vegetables,13
5,24852,Banana,fresh fruits,13
6,47626,Large Lemon,fresh fruits,12
7,27845,Organic Whole Milk,milk,11
8,4605,Yellow Onions,fresh vegetables,9
9,4799,Shredded Parmesan,packaged cheese,8
